In [8]:
from parser import *
from config_parser import *
import re

In [2]:
# Собираем 300 товаров (3 страницы)
# api_df = get_wb_products(query="мио стимулятор",pages=3) Для тестов 68 записей
api_df = get_wb_products(query="миостимулятор",pages=100)
api_df.to_csv("../data/wb_products.csv", index=False)
print(f"Собрано {len(api_df)} товаров")

Страница 1 получена
Страница 2 получена
Страница 3 получена
Страница 4 получена
Страница 5 получена
Страница 6 получена
Страница 7 получена
Страница 8 получена
Страница 9 получена
Страница 10 получена
Страница 11 получена
Страница 12 получена
Страница 13 получена
Страница 14 получена
Страница 15 получена
Страница 16 получена
Страница 17 получена
Страница 18 получена
Страница 19 получена
Страница 20 получена
Страница 21 получена
Страница 22 получена
Страница 23 получена
Страница 24 получена
Страница 25 получена
Страница 26 получена
Страница 27 получена
Страница 28 получена
Страница 29 получена
Страница 30 получена
Страница 31 получена
Страница 32 получена
Страница 33 получена
Страница 34 получена
Страница 35 получена
Страница 36 получена
Страница 37 получена
Страница 38 получена
Страница 39 получена
Страница 40 получена
Страница 41 получена
Страница 42 получена
Страница 43 получена
Страница 44 получена
Страница 45 получена
Страница 46 получена
Страница 47 получена
Страница 48 получена
С

In [3]:
api_df = pd.read_csv("../data/wb_products.csv")
api_df

,id,name,price,rating,feedbacks,brand
0,397455530,Массажер миостимулятор для тела пресса спины ш...,1278.0,4.7,71,eFlow
1,221329577,Миостимулятор 36 программ антицеллюлитный для ...,2052.0,4.9,1021,NaN
2,334678554,Миостимулятор 48 программ массажер для похудения,4196.0,4.9,1021,NaN
3,359420749,Миостимулятор для тела антицеллюлитный для пох...,1945.0,4.9,13,NaN
4,221352035,Миостимулятор 36 программ массажер для похудения,1847.0,4.9,1021,NaN
...,...,...,...,...,...,...
5995,384483737,Косметологический аппарат B-333,11551.0,0.0,0,NaN
5996,248314031,Массажер FH-320 цвета в ассортименте,1332.0,0.0,0,Isa
5997,237927173,Массажер для шеи и плеч KL-5830 микротоковый,1839.0,0.0,0,Neck Massager
5998,237927166,Массажер для шеи и плеч KL-5830,2016.0,0.0,0,Neck Massager


In [6]:
# Первичный фильтр
ban_word = ["электрод"]
ban_pattern = "|".join(ban_word)

clear_df = api_df[api_df["name"].str.contains(ban_pattern, case=False, na=False)]
clear_df.to_csv("c.csv")

In [47]:
# Первичный фильтр 

# Ключевые слова для ВКЛЮЧЕНИЯ
# include_keywords = [
#     r"\bмиостимулятор\b", r"\bэлектростимулятор\b", 
#     r"\bems\b", r"\bнейростимулятор\b", r"\bэлектронный стимулятор\b",
#     r"\bэлектрический стимулятор\b", r"\bимпульсный стимулятор\b",
#     r"\bмиостимуляция\b"
# ]

# Ключевые слова для ИСКЛЮЧЕНИЯ (аксессуары и нерелевантные категории)
# exclude_patterns = [
#     r"электрод", r"шнур", r"кабель", r"провод", 
#     r"пластин", r"насадк", r"гель", r"коврик", 
#     r"пульт", r"крепеж", r"держатель", r"запчасть",
#     r"аксессуар", r"электромассажер", r"вакуумный",
#     r"перкуссионный", r"вибромассажер", r"ручной",
#     r"сменные", r"аппликатор", r"бандаж", r"маска",
#     r"светодиодн", r"led", r"роликовый", r"массажер[^,]*$"
# ]

exclude_patterns = [
    r"электрод", r"шнур", r"кабель", r"провод", 
    r"пластин", r"гель",
    r"крепеж", r"держатель", r"запчасть",
    r"аксессуар", r"\bLED\b"
]

# Фильтрация нерелевантных товаров
def is_relevant(name):
    if pd.isna(name):
        return False
        
    name = name.lower()
    
    # Проверка на исключающие термины
    if any(re.search(pattern, name) for pattern in exclude_patterns):
        return False
        
    # Проверка на включающие термины
    # return any(re.search(kw, name) for kw in include_keywords)
    return True

mask = api_df['name'].apply(is_relevant)
clear_df = api_df[mask].copy()
# mask

# 1. Удалить товары без отзывов
clear_df = clear_df[clear_df['feedbacks'] > 0]

# 2. Рассчитать пороги для ценовых сегментов
price_25 = clear_df['price'].quantile(0.25)
price_75 = clear_df['price'].quantile(0.75)
feedback_median = clear_df['feedbacks'].median()

# 3. Отобрать топ-20% по отзывам
top_20_percent = clear_df[clear_df['feedbacks'] >= clear_df['feedbacks'].quantile(0.8)]

# 4. Сбалансированные выборки по ценовым сегментам
low_price = clear_df[clear_df['price'] < price_25].nlargest(30, 'feedbacks')
mid_price = clear_df[(clear_df['price'] >= price_25) & (clear_df['price'] <= price_75)].nlargest(30, 'feedbacks')
high_price = clear_df[clear_df['price'] > price_75].nlargest(30, 'feedbacks')

# 5. Недооцененные товары с высоким рейтингом
undervalued = clear_df[
    (clear_df['rating'] >= 4.8) & 
    (clear_df['price'] < clear_df['price'].median()) & 
    (clear_df['feedbacks'] < feedback_median)
].nlargest(20, 'rating')

# Объединить все отобранные товары
final_ids = pd.concat([
    top_20_percent,
    low_price,
    mid_price,
    high_price,
    undervalued
])['id'].drop_duplicates()

clear_df = api_df[api_df["id"].isin(final_ids)]

clear_df.to_csv("c.csv")

In [47]:
driver = init_driver_firefox(headless=browser_headless)
# driver = init_driver_firefox()

In [ ]:
final_main_df = pd.DataFrame(columns=['id', 'power_type', 'zones', 'description'])
final_specs_df = pd.DataFrame(columns=['good_id', 'group_name', 'name', 'value'])
curr_count = 0
total = len(api_df)

for product_id in api_df["id"]:
    try:
        curr_count += 1
        # Получаем данные товара (предполагаем, что функция get_product_details реализована)
        details = get_product_details(driver, product_id)
        
        # Парсим данные товара
        current_main_df, current_specs_df = parse_product_data(details, product_id)
        
        # Добавляем данные в итоговые DataFrame
        final_main_df = pd.concat([final_main_df, current_main_df], ignore_index=True)
        final_specs_df = pd.concat([final_specs_df, current_specs_df], ignore_index=True)

        print(f"Получение товара {product_id} выполнено; Осталось получить: {total - curr_count} ")
        delay = random.uniform(1, 3)
        time.sleep(delay)


        
    except Exception as e:
        print(f"==================================\nОшибка при обработке товара {product_id}: {str(e)}\n==================================")
        continue


Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание 1 не найдено
Описание успешно записано
Получение товара 291888914 выполнено; Осталось получить: 67 
Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание 1 не найдено
Описание успешно записано
Получение товара 143533250 выполнено; Осталось получить: 66 
Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание 1 не найдено
Описание успешно записано
Получение товара 174098651 выполнено; Осталось получить: 65 
Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание успешно записано
Получение товара 44081627 выполнено; Осталось получить: 64 
Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание успешно записано
Получение товара 236798629 выполнено; Осталось получить: 63 
Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание 1 не найдено
Описание успешно записано
Получение товара 176

In [ ]:
merged_main_df = pd.merge(
    api_df,
    final_main_df,
    left_on="id",
    right_on="id",
    how='left'
)
merged_main_df

,id,name,price,rating,feedbacks,brand,power_type,zones,description
0,291888914,Спортивный массажер миостимулятор для тела эле...,2607.0,4.7,2160,TARGO,от аккумулятора; от сети,живот; ягодицы; бедра,Спортивный массажер мио стимулятор - многофунк...
1,143533250,"Массажер миостимулятор для тела, спины, шеи и ...",2677.0,4.7,2160,TARGO,от аккумулятора; от сети,спина; тело; ноги,Массажер мио стимулятор - многофункциональный ...
2,174098651,Массажер мио стимулятор для шеи,5400.0,4.3,10,,встроенный аккумулятор,шея,Мио стимулятор для шеи - ощущение комфорта и р...
3,44081627,Миостимулятор для пресса тренажер ABS Trainer MIO,5012.0,4.7,1499,Yamaguchi,5 В; от аккумулятора; без питания,пресс; живот; тело,Миостимулятор для тела Yamaguchi ABS Trainer M...
4,236798629,Миостимулятор массажер для терапии,632.0,4.8,30,Другой мир,от сети; USB,плечи; спина; тело,Массажер электрический для лица и тела. Миости...
...,...,...,...,...,...,...,...,...,...
63,232932984,Массажёр с улучшенной функцией стимуляции,1163.0,0.0,0,KarmiHome,None,None,Дорогие клиенты! Мы хотим сказать огромное спа...
64,232932987,Массажёр с комбинированным воздействием,1164.0,0.0,0,KarmiHome,None,None,Дорогие клиенты! Мы хотим сказать огромное спа...
65,212473759,Антицеллюлитный массажер для тела миостимулятор,682.0,3.9,8,М.А.Г.,None,живот; плечи; тело,Миостимулятор для пресса мышц живота это небол...
66,290768035,Массажный мио стимулятор коврик для стоп,368.0,1.5,2,BigMarket,None,None,Акупунктурный массажер — это современный инстр...


In [12]:
merged_main_df.to_csv("final_wb_products.csv", index=False)

In [14]:
final_specs_df.to_csv("final_other_specs.csv", index=False)

In [9]:
final_specs_df

,good_id,group_name,name,value
0,291888914,Основная информация,Цвет,белый
1,291888914,Общие характеристики,Модель,беспроводной
2,291888914,Общие характеристики,Гарантийный срок,1 год
3,291888914,Питание,Питание,от аккумулятора; от сети
4,291888914,Технические особенности,Количество режимов работы,15 режимов
...,...,...,...,...
966,301536327,Дополнительная информация,Срок годности,3 года
967,301536327,Дополнительная информация,Комплектация,инозитол 60 капсул; Хром 90 капсул
968,301536327,Дополнительная информация,Максимальная температура хранения,+25 °C
969,301536327,Дополнительная информация,Торговое наименование,"биологически активная добавка к пище ""Инозитол..."


In [ ]:
# product_id = api_df.iloc[0]["id"]
details = get_product_details(driver, 176410178)

Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание 1 не найдено
Описание успешно записано


In [6]:
print(details)

{'description': 'Мио стимулятор для шеи - ощущение комфорта и расслабления на грани нового уровня!Приветствую! Если ты ищешь идеальное средство для максимального расслабления и ухода за своим телом, то Массажер мио стимулятор для шеи - твой идеальный выбор! Позволь мне рассказать тебе, почему этот микротоковый миостимулятор стоит твоего внимания.Помимо уникального дизайна и компактных размеров, этот электрический миостимулятор специально разработан для эффективного массажа мышц шеи, спины и воротниковой зоны. Он использует микротоки, чтобы активировать мышцы, способствуя глубокому расслаблению и оздоровлению всего тела.Вот почему Массажер электростимулятор для шеи - это революционный аппарат для миостимуляции и электрического массажа:- Миостимуляция: Уникальная технология микротоков позволит тебе нежно стимулировать мышцы, обеспечивая непревзойденное чувство расслабления и снятия напряжения.- Легкий и компактный: Благодаря своему мини размеру, этот массажер станет твоим незаменимым спу

In [ ]:
main_specs_df, other_specs_df = parse_product_data(details, 176410178)

In [48]:
# Парсинг отзывов
# https://www.wildberries.ru/catalog/291888914/feedbacks 

# Передаём драйвер как аргумент
import re

def get_product_feedbacks(driver, product_id):
    driver.get(f"https://www.wildberries.ru/catalog/{product_id}/feedbacks")

    # Ожидание загрузки основного контейнера с отзывами
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "comments__list, .non-comments"))
    )
    
    try:
        no_feedback = driver.find_element(By.CLASS_NAME, ".non-comments")
        print(f"{product_id} - нет отзывов")
        return pd.DataFrame(columns=['product_id', 'rating', 'advantage', 'disadvantage', 'comment'])
    except:
        pass

    # Проверка и переключение на вкладку "Этот вариант" если доступна
    try:
        # Ожидаем появления переключателя вариантов
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".product-feedbacks__tabs"))
        )
        
        # Ищем кнопку "Этот вариант"
        variant_button = driver.find_element(
            By.CSS_SELECTOR, "li.product-feedbacks__tab:nth-child(2) > button:nth-child(1)"
        )
        variant_button.click()
        time.sleep(1.5)
    except:
        # Если нет переключателя или кнопки, продолжаем как обычно
        print(f"{product_id} - не удалось найти кнопку \"Этот вариант\"")
        pass

    # Прокрутка страницы для загрузки ВСЕХ отзывов
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0
    max_scroll_attempts = 100 # Максимум попыток прокрутки для защиты от бесконечного цикла

    while scroll_attempts < max_scroll_attempts:
        # Прокрутка вниз
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)  # Ожидание подгрузки контента
        
        # Проверка изменения высоты страницы
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        scroll_attempts += 1

    # Сбор всех отзывов
    feedback_items = driver.find_elements(By.CSS_SELECTOR, "li.comments__item.feedback")
    feedbacks_list = []

    for item in feedback_items:
        try:
            # Парсинг рейтинга
            rating_elem = item.find_element(By.CLASS_NAME, "feedback__rating")
            rating_class = rating_elem.get_attribute("class")
            rating = int(re.search(r'star(\d+)', rating_class).group(1))
        except:
            rating = None

        # Инициализация текстовых полей
        advantage = None
        disadvantage = None
        comment = None
        
        # Парсинг текста отзыва
        try:
            text_block = item.find_element(By.CSS_SELECTOR, ".feedback__text.j-feedback__text")
            
            # Обработка структурированных отзывов (с разделами)
            sections = text_block.find_elements(By.CLASS_NAME, "feedback__text--item")
            if sections:
                for section in sections:
                    text = section.text.strip()
                    if not text:
                        continue
                        
                    if "feedback__text--item-pro" in section.get_attribute("class"):
                        advantage = text
                    elif "feedback__text--item-con" in section.get_attribute("class"):
                        disadvantage = text
                    else:
                        comment = text
            # Обработка неструктурированных отзывов
            else:
                comment = text_block.text.strip()
        except:
            pass  # Если текста нет, оставляем поля пустыми

        feedbacks_list.append({
            'product_id': product_id,
            'rating': rating,
            'advantage': advantage,
            'disadvantage': disadvantage,
            'comment': comment
        })

    # Создание DataFrame
    feedbacks = pd.DataFrame(feedbacks_list)
    print(f"{product_id} Отзывы успешно собраны. Количество отзывов: {len(feedbacks)}")
    return feedbacks

In [50]:
feedback = get_product_feedbacks(driver, 143533250)

feedback

143533250 Отзывы успешно собраны. Количество отзывов: 844


,product_id,rating,advantage,disadvantage,comment
0,143533250,5,Достоинства:Всё понравилось,"Недостатки:Не выключается экран при зарядке, в...",Комментарий:Аппарат брала для похудения и тону...
1,143533250,5,"Достоинства:Зарядки хватает на долго, пользуем...",None,None
2,143533250,5,Достоинства:Отличный прибор,None,None
3,143533250,5,"Достоинства:Прибор отличный, каждый день с жен...",None,None
4,143533250,5,Достоинства:Всё отлично,None,None
...,...,...,...,...,...
839,143533250,5,None,None,Отлично работает хорошо
840,143533250,5,None,None,🔥🔥🔥 от усталости. Жена радуется как ребенок) Д...
841,143533250,5,None,None,Восторг! Спина ликует! Даже над коленками накл...
842,143533250,5,None,None,Все работает. Все чудесно. Мощный.


In [51]:
feedback.to_csv("feedback.csv")

In [ ]:
driver.quit()  # Закрываем браузер только после всех запросов

Тестыыыы

{'description': 'Электростимулятор чрескожный универсальный Нейродэнс ПКМ — это современный физиотерапевтический аппарат, который станет надежным помощником в восстановлении и поддержании здоровья. Он предназначен для применения в домашних условиях, предоставляя возможность получать эффективные физиотерапевтические процедуры без необходимости посещения медицинских учреждений.\n\nПрибор работает на основе технологии электростимуляции, что позволяет воздействовать на мышцы, улучшая кровообращение и облегчая болевые ощущения. Устройство может использоваться для различных целей: от снятия напряжения после трудного дня до реабилитации после травм. С ним вы сможете проводить высококачественные процедуры, направленные на восстановление функциональности мышц и суставов, а также на долголетие здоровья.\n\nУниверсальный электростимулятор обладает несколькими режимами работы, которые легко переключаются в зависимости от потребностей пользователя. Компактный размер и стильный дизайн устройства позволяют его удобно использовать как в домашних условиях, так и брать с собой в поездки. Медицинский прибор оснащен простым и интуитивно понятным управлением, что делает его доступным для всех возрастных категорий.\n\nБлагодаря высококачественным материалам и современным технологиям, этот физиотерапевтический аппарат обеспечивает надежность и долгий срок службы. Нейродэнс ПКМ идеально подходит для тех, кто заботится о своем здоровье и стремится к активной жизни.\n\nС этим электростимулятором вы можете легко осуществлять заботу о своем организме, не выходя из дома. Он станет надежным партнёром в процессе восстановления и улучшения самочувствия.', 
'specifications': 
{'Общие характеристики': {'Гарантийный срок': '1 год'}, 
'Питание': {'Питание': 'от батареек'}, 
'Технические особенности': {'Количество режимов работы': '24'}, 
'Материалы': {'Материал корпуса': 'пластик'}, 
'Дополнительная информация': {'Вид воздействия': 'ток низкой частоты на определённые зоны тела', 
'Область применения': 'для всего тела', 'Противопоказания': 'индивидуальная непереносимость; наличие имплантированного кардиостимулятора', 'Комплектация': 'батарейки АА (2шт.); инструкция по эксплуатации; электростимулятор', 'Страна производства': 'Россия'}, 
'Габариты': {'Высота предмета': '4.5 см', 'Ширина предмета': '5.5 см', 'Длина предмета': '14.5 см', 'Вес товара без упаковки (г)': '350 г', 'Вес товара с упаковкой (г)': '425 г', 'Длина упаковки': '24 см', 'Высота упаковки': '9 см', 'Ширина упаковки': '17 см'}, 
'Документы': {'Регистрационное удостоверение': 'РЗН 2019/9330'}, 
'Без названия': {'': ''}}, 
'power_type': 'от батареек'}

{'description': 'Мио стимулятор для шеи - ощущение комфорта и расслабления на грани нового уровня!Приветствую! Если ты ищешь идеальное средство для максимального расслабления и ухода за своим телом, то Массажер мио стимулятор для шеи - твой идеальный выбор! Позволь мне рассказать тебе, почему этот микротоковый миостимулятор стоит твоего внимания.Помимо уникального дизайна и компактных размеров, этот электрический миостимулятор специально разработан для эффективного массажа мышц шеи, спины и воротниковой зоны. Он использует микротоки, чтобы активировать мышцы, способствуя глубокому расслаблению и оздоровлению всего тела.Вот почему Массажер электростимулятор для шеи - это революционный аппарат для миостимуляции и электрического массажа:- Миостимуляция: Уникальная технология микротоков позволит тебе нежно стимулировать мышцы, обеспечивая непревзойденное чувство расслабления и снятия напряжения.- Легкий и компактный: Благодаря своему мини размеру, этот массажер станет твоим незаменимым спутником в путешествиях и повседневной жизни. Наслаждайся массажем в любое время и в любом месте!- Беспроводной: Этот электромиостимулятор оснащен батареей, поэтому нет необходимости использовать провода. Готов к использованию в любой момент!- Многофункциональный: Не только для шеи! Этот электромассажер также отлично подходит для использования на других зонах тела, таких как воротниковая зона, часть мышц спины. Обнаружь новые возможности для ухода за собой!- Оздоровление и косметический эффект: Благодаря электростимуляции мио стимулятор поможет усилить массажный эффект и улучшить общее состояние кожи. Получи не только расслабление, но и красоту!Если ты ищешь идеальный подарок для себя или для близкого человека, то Массажер Мио стимулятор для шеи - это идеальный выбор. Не упусти возможность насладиться максимальным расслаблением и оздоровительным эффектом, просто нажми на кнопку "Добавить в корзину" прямо сейчас!', 'specifications': {'Основная информация': {'Цвет': 'зеленый; черный'}, 'Общие характеристики': {'Количество предметов в упаковке': '1 шт.'}, 'Материалы': {'Материал изделия': 'пластик'}, 'Дополнительная информация': {'Зона массажа': 'шея; воротниковая зона', 'Действие': 'микротоки; импульсы; стимулирует кровообращение и улучшает питание тканей', 'Страна производства': 'Китай', 'Комплектация': 'массажер; usb-кабель'}, 'Габариты': {'Ширина предмета': '13 см', 'Высота предмета': '15 см', 'Вес товара с упаковкой (г)': '0.3 г', 'Длина упаковки': '19 см', 'Высота упаковки': '15 см', 'Ширина упаковки': '5 см'}}, 'power_type': None, 'zones': 'шея; воротниковая зона'}

In [ ]:

# https://basket-18.wbbasket.ru/vol2918/part291888/291888914/info/ru/card.json опицональный вариант парсинга инфы о товаре


In [63]:
# random.uniform(0.1, 3)

for i in range(100):
    print(random.uniform(0.5, 1.5))

0.5935708008896233
0.9528682536263479
0.7281459139371663
0.7212937024140541
1.3612180180114044
1.3343318902111054
0.9688114318942107
1.1060621233017311
0.9258313495331713
1.2549722835066326
1.0985746803036114
1.1769997141033617
0.6039166566408797
0.7452885978940176
0.9480435178253672
0.6385073295711176
0.6124980923165215
1.2322159356590725
0.7248834323040282
0.7749586224226347
1.2988430539220255
0.8912295555059521
0.6567131591982344
0.6477734007223448
1.1024019075743505
0.851992761472716
0.5772010905996738
0.5630929824044376
0.709656650910031
1.404011351241797
0.7297474271058513
1.2325315873842713
0.8538759285494836
1.410908781719846
0.5243524410581698
1.0509435501482942
1.2465779264823658
0.7062496901543402
0.6474919983043484
0.5286273365971108
1.1119429716332494
1.3349764356906395
1.2774549954365433
0.5191761150363726
1.3683097808987947
1.404062209868552
0.8598273662449979
0.829103321438213
0.6593733630287486
0.6575632010517043
1.0032455923588754
0.72941721687951
1.154264812182317
0.